In [1]:
import requests 
import nltk
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import numpy as np
import math
from IPython.display import Markdown, display
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt    
from dataHack import removeColl 
%matplotlib inline
import warnings
import scipy
warnings.filterwarnings('ignore')
import networkx as nx
%load_ext autotime
def printmd(string):
    display(Markdown(string))
from selenium.webdriver.common.keys import Keys

In [2]:
data= pd.read_csv('train.tsv',sep='\t')

time: 221 ms


In [3]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


time: 165 ms


#### For every SentenceId there are possible derived phrases labelled with Sentiment. 

In [7]:
sentiment_dict={0 : 'negative',
                1 : 'somewhat negative',
                2 : 'neutral',
                3 : 'somewhat positive',
                4 : 'positive'}

time: 3 ms


In [18]:
printmd('### No. of sentences - '+ str(data['SentenceId'].nunique()))

### No. of sentences - 8529

time: 8 ms


In [55]:
data[data['SentenceId']==8545]

,PhraseId,SentenceId,Phrase,Sentiment


time: 9 ms


In [47]:
test= pd.read_csv('test.tsv',sep='\t')

time: 99 ms


In [48]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


time: 15 ms


In [58]:
from nltk.corpus import stopwords
import re
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

time: 15 ms


In [59]:
data['Phrase']=data['Phrase'].apply(lambda l:review_to_words(l) )

time: 2min 32s


In [66]:
test['Phrase']=test['Phrase'].apply(lambda l:review_to_words(l) )

time: 1min 10s


In [61]:
words= pd.DataFrame(' '.join(data['Phrase'].tolist()).split())

print words.info()

printmd( '### '+str(words[0].nunique()))

del words

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628973 entries, 0 to 628972
Data columns (total 1 columns):
0    628973 non-null object
dtypes: object(1)
memory usage: 4.8+ MB
None


### 14993

time: 300 ms


In [71]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 9000) 
train_data_features = vectorizer.fit_transform(data['Phrase'])

Creating the bag of words...

time: 2.76 s


In [72]:
#train_data_features=train_data_features.toarray()

time: 2 ms


In [ ]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["Sentiment"] )

Training the random forest...


In [ ]:
print "Creating the bag of words...\n"

test_data_features = vectorizer.fit_transform(test['Phrase'])

In [ ]:
#test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"PhraseId":test["PhraseId"], "Sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Rotten1.csv", index=False, quoting=3 )